In [4]:
def cond():
    x = 3
    if x < 5:
        return 'yes'
    else:
        return 'no'
    
cc = cond.__code__.co_code
print(list(cond.__code__.co_code))

import dis
dis.dis(cond)

print('100 in op is:', dis.opname[100])

[100, 1, 125, 0, 124, 0, 100, 2, 107, 0, 114, 16, 100, 3, 83, 0, 100, 4, 83, 0, 100, 0, 83, 0]
  2           0 LOAD_CONST               1 (3)
              2 STORE_FAST               0 (x)

  3           4 LOAD_FAST                0 (x)
              6 LOAD_CONST               2 (5)
              8 COMPARE_OP               0 (<)
             10 POP_JUMP_IF_FALSE       16

  4          12 LOAD_CONST               3 ('yes')
             14 RETURN_VALUE

  6     >>   16 LOAD_CONST               4 ('no')
             18 RETURN_VALUE
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE
100 in op is: LOAD_CONST


### In Python3, bytecode are printed as numbers.

```
>>> list(cond.__code__.co_code)  # the bytecode as numbers
[100, 1, 0, 125, 0, 0, 124, 0, 0, 100, 2, 0, 107, 0, 0, 114, 22, 0, 100, 3, 0, 83, 
 100, 4, 0, 83, 100, 0, 0, 83]
```

The second and third bytes—1, 0—are arguments to LOAD_CONST, 

while the fifth and sixth bytes—0, 0—are arguments to STORE_FAST

## Why use two bytes for each argument?

If Python used just one byte to locate constants and names instead of two, you could only have 256 names/constants associated with a single code object. 

Using two bytes, you can have up to **256 squared**, or 65,536

## jump --> POP_JUMP_IF_FALSE

This instruction will pop the top value off the interpreter's stack. 

If the value is true, then nothing happens. (The value can be "truthy"—it doesn't have to be the literal True object.) 

If the value is false, then the interpreter will jump to another instruction.

# Homework

1. What's the difference between a for loop and a while loop to the Python interpreter?
2. How can you write different functions that generate identical bytecode?
3. How does elif work? What about list comprehensions?

# Frame

A frame is a collection of information and context for a chunk of code. 

Frames are created and destroyed on the fly as your Python code executes. 

There's one frame corresponding to each call of a function

```
>>> def bar(y):
...     z = y + 3     # <--- (3) ... and the interpreter is here.
...     return z
...
>>> def foo():
...     a = 1
...     b = 2
...     return a + bar(b) # <--- (2) ... which is returning a call to bar ...
...
>>> foo()             # <--- (1) We're in the middle of a call to foo ...
3
```

![Call stack](img/callstack.png)

There are data block, we used for pop/push variables.

Call stack.

As well as data stack: used for certain kinds of control flow, particularly looping and exception handling.

#### Each frame on the call stack has its own data stack and block stack.

# Byterun

In [7]:
import dis
dis.HAVE_ARGUMENT
dis.hasconst

[dis.opname[op] for op in dis.hasname]
[dis.opname[op] for op in dis.haslocal]

[dis.opname[op] for op in dis.hasjrel]

['FOR_ITER',
 'JUMP_FORWARD',
 'SETUP_LOOP',
 'SETUP_EXCEPT',
 'SETUP_FINALLY',
 'SETUP_WITH',
 'SETUP_ASYNC_WITH']

# Real shit!

In [35]:
class VirtualMachine(object):
    def __init__(self):
        self.frames = []
        self.frame = None
        self.return_value = None
    
    def run_code(self, code, global_names=None, local_names=None):
        # make frame
        frame = self.make_frame(code, 
                                global_names=global_names, 
                                local_names=local_names)
        
        self.run_frame(frame)
        
    def push_frame(self, frame):
        self.frames.append(frame)
        self.frame = frame
    
    def pop_frame(self):
        self.frames.pop()
        if self.frames:
            self.frame = self.frames[-1]
        else:
            self.frame = None
    
    def parse_byte_and_args(self):
        f = self.frame
        pc =f.pc
        byte_code = f.code_obj.co_code[pc]
        byte_name = dis.opname[byte_code]
        print(byte_code, byte_name)
        
        f.pc += 1
        argument = []
        return byte_name, argument
    
    def dispatch(self, byte_name, argument):
        pass
    
    
    def run_frame(self, frame):
        self.push_frame(frame)
        while 1:
            byte_name, args = self.parse_byte_and_args()
            
            why = self.dispatch(byte_name, args)
        
            if why:
                break # how to break?
                
        
        self.pop_frame()
        
        return self.return_value
    
    def make_frame(self, code, callargs={}, global_names=None, local_names=None):
        #local_names.update(callargs)
        frame = Frame(code, global_names, local_names, self.frame)
        return frame
        
class Frame(object):
    def __init__(self, code_obj, global_names, local_names, prev_frame):
        self.code_obj = code_obj
        self.stack = []
        self.pc = 0
        self.prev_frame = prev_frame
        

code = """
a = 3
b = a
"""
code = textwrap.dedent(code)
code = compile(code, "<123>", "exec", 0, 1)

vm = VirtualMachine()  
vm.run_code(code)
    

100 LOAD_CONST
0 <0>
90 STORE_NAME
0 <0>
101 LOAD_NAME
0 <0>
90 STORE_NAME
1 POP_TOP
100 LOAD_CONST
1 POP_TOP
83 RETURN_VALUE
0 <0>


IndexError: index out of range

In [37]:
import textwrap

code = """
a = 3
b = a
"""
code = textwrap.dedent(code)
code = compile(code, "<123>", "exec", 0, 1)
print(list(code.co_code))

print([dis.opname[op] for op in list(code.co_code)])

dis.dis(code)

[100, 0, 90, 0, 101, 0, 90, 1, 100, 1, 83, 0]
['LOAD_CONST', '<0>', 'STORE_NAME', '<0>', 'LOAD_NAME', '<0>', 'STORE_NAME', 'POP_TOP', 'LOAD_CONST', 'POP_TOP', 'RETURN_VALUE', '<0>']
  2           0 LOAD_CONST               0 (3)
              2 STORE_NAME               0 (a)

  3           4 LOAD_NAME                0 (a)
              6 STORE_NAME               1 (b)
              8 LOAD_CONST               1 (None)
             10 RETURN_VALUE
